<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">




<br><br><br>
<h2><font color="#00586D" size=4>Capstone III</font></h2>

<h1><font color="#00586D" size=5>Exploración sobre la base de datos de películas</font></h1>
<br><br><br>


<div align="right">
<font color="#00586D" size=3>Luis de la Ossa</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>
<br>

---
<br>

En este proyecto se parte de un conjunto de datos sobre películas y valoraciones sobre las mismas disponibles en _Internet Movie Database (IMDB)_. Esta información se ha enriquecido con datos adicionales obtenidos mediante consultas a la API de otro sitio, _The Movie Database (TMDB)_. Como resultado de este proceso, se han generado  tres conjuntos de datos: `df_movies`, con información sobre películas, géneros, y valoraciones;  `df_people`, con información sobre las personas; y `df_credits` con información sobre las personas que participan en cada película (director y tres actores principales). A partir de éstos, se llevará a cabo un análisis exploratorio.
  
El modo en que se desarrolla el análisis depende en gran medida _de los datos disponibles_ y de unos _objetivos_ que pueden ser generales, o la respuesta a cuestiones concretas, y que pueden variar o ampliarse en función de los resultados intermedios que vaya arrojando el proceso. En este proyecto se partirá de una situación hipotética inicial en la que se estudian las condiciones para la inversión en una película de _presupuesto relativamente bajo_. Para ello, se analizarán qué factores tienen más incidencia en el éxito de una película, que consideraremos que puede cuantificarse por los ingresos que ha obtenido, o por el número de votos y valoración en _IMDB_. También se estudiarán las diferencias entre géneros con el fin de seleccionar los más adecuados, y se analizará la relevancia de directores, actores, y actrices, y cuales podrían ser los más adecuados. Además, y paralelamente, se obtendrá alguna información adicional que, si bien puede no estár relacionada con el objetivo principal, puede resultar de interés. 

Para llevar a cabo la exploración, se propondrán una serie de ejercicios que consisten en la obtención de gráficas o datos concretos. Por último, este proyecto no tiene porqué ceñirse a la resolución de los ejercicios, sino que cabe la posibilidad de dar respuesta a cuestiones distintas a las propuestas en la libreta.



<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Una parte esencial de la exploración es la validación y limpieza de los datos. Como esta parte es más "árida", en este proyecto nos centraremos en la exploración propiamente dicha. En el Capstone V veremos un caso en el que la preparación de los datos es algo más compleja.
</div>

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>


* [1. Primera inspección](#section1)
  * [1.1 Exploración inicial de `df_movies`](#section11)
* [2. Presupuesto](#section2)
  * [2.1 Presupuesto / ingresos](#section21)
  * [2.2 Fecha de estreno como factor de interés](#section22) 
* [3 Géneros](#section3)  
* [4 Personas](#section4)  
  * [4.1 Directores](#section41)
  * [4.2 Reparto](#section42) 
* [5 Películas de bajo presupuesto](#section5)  
* [6. Conclusión](#section6)

---

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section1"></a>
## <font color="#00586D"> 1. Primera inspección</font>
<br>

El primer paso en el análisis exploratorio consiste en obtener una primera descripción del conjunto de datos como tal, y que incluye aspectos como: archivos; características, relaciones entre ellas, significado y tipos de datos con que se representan; existencia o frecuencia de valores perdidos, etc. A continuación se lee y se muestra la cabecera de cada uno de los tres *DataFrame*, y se inspecciona su estructura.


#### `df_movies`

In [ ]:
df_movies = pd.read_csv('data/df_movies.csv', index_col='movie_id', sep=';', parse_dates=['release_date'])
display(df_movies.head(2))
df_movies.info()

Puede apreciarse que todos los tipos de datos son los esperados, y que aparentemente no hay datos perdidos salvo en el caso de las palabras clave (que no se utilizarán en este proyecto).

#### `df_people`

In [ ]:
df_people = pd.read_csv('data/df_people.csv', index_col='people_id', sep=';')
display(df_people.head(2))
df_people.info()

Los datos corresponden también a lo esperado, y que existen valores perdidos para la columna `imdb_id`. Se sobreentiende que algunos de los directores/actores no tienen identificador en la plataforma `IMDb`.

#### `df_credits`

In [ ]:
df_credits = pd.read_csv('data/df_credits.csv', sep=';')
display(df_credits.head(2))
df_credits.info()

Puede observarse que tampoco existen valores perdidos en este conjunto, y que la codificación de los datos es correcta.


Como consecuencia de esta primera aproximación, se concluye que no es necesario hacer ningún tipo de preprocesamiento, ni adaptar el proceso a la limpieza de datos, por lo que la exploración se centrará en la información.

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section11"></a>
### <font color="#00586D"> 1.1 Exploración inicial de `df_movies` </font>
<br>

El *DataFrame* `df_movies` es el núcleo del conjunto de datos, y entorno al cual debe desarrollarse la exploración. Contiene información identificativa, cualitativa, y cuantitativa sobre cada película. En primer lugar se hará una inspección de los datos para comprobar si existen valores extremos.

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 1</font>

Obtener una descripción de las columnas numéricas del *DataFrame* `df_movies`. Con el fin de observar si existen valores extremos y en qué proporción, incorporar a los resultados los cuantiles `[0.25, 0.5, 0.75, 0.95, 0.99, 0.995]`.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Se puede utilizar `DataFrame.describe()`

</details>  

In [ ]:
# Completar

<!--comment -->

Algo que se observa es que existen películas para las que el presupuesto y/o la recaudación son cero, lo que realmente corresponde a un valor perdido. Se van a tratar de forma distinta ambas columnas. Puesto que la recaudación es el factor de más interés, se eliminarán las películas para las que no la conozcamos. 

Eliminar las películas cuyo valor `revenue` es cero (seleccionar aquellas cuya recaudación es mayor que cero).

In [ ]:
df_movies = # Completar

<!--comment -->

 Por otra parte, se visualizan las películas en las que los presupuestos que sean igual a cero.

In [ ]:
# Completar

<!--comment -->

En principio parecen películas que han recibido un gran número de votos, y algunas de ellas han conseguido una gran recaudación, por lo que no se descartarán. Se codificarán como *NaN* para tratarlos adecuadamente llegado el caso.

Asignar el valor *NaN* a los valores que son cero en la columna `df_movies['budget']`. 

In [ ]:
# Completar

<!--comment -->

Por otra parte, si bien los rangos de las variables son muy amplios en algunos casos, se aprecia una diferencia importante entre el valor **máximo** de la columna `popularity` y el resto. Es necesario por tanto determinar si se trata de un error, o por el contrario existen valores fuera de rango, pero que deben ser considerados dentro del estudio.

Mostrar las 10 filas de `df_movies` con mayor valor de popularidad.

In [ ]:
# Completar

<!--comment -->

La visualización de los datos permite intuir que la mayoría de las películas para las que el valor de la columna `popularity` es extremadamente alto, son conocidas y de alto presupuesto. Llama la atención también un fenómeno, y es que gran parte de ellas se estrenaron el año pasado o éste. Eso lleva a pensar si el indicador podría estar relacionado con una mayor actividad en el sitio web fruto del uso cada vez más intensivo de las redes sociales; o como en el caso de  otras plataformas (*Spotify*, por ejemplo) es un indicador variable que refleja el interés en el momento actual. Esta circunstancia puede comprobarse mostrando la popularidad media de las películas por año.


Existen diversas formas de mostrar la relación entre popularidad y fecha de estreno, una de ellas es mediante un gráfico de barras. Sin embargo, no es la más conveniente debido a que se está contemplando un periodo de casi 100 años y sería más difícil identificar tendencias. Dibujar una gráfica de tipo `sns.lineplot()` que muestre la popularidad media por año. 

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* `sns.lineplot()` muestra, para cada valor del eje *x*, la media e intervalo de confianza (obtenido mediante *bootstraping*) de los valores del eje *y* correspondientes.
* Debe pasarse a la función, además de la columna `popularity`, la columna correspondiente a los años de estreno de cada película, accesible mediante `Series.dt.year`.
</details>  


In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Efectivamente, se muestra que la popularidad de las películas crece  con la fecha del estreno, y que se produce un aumento dramático a partir del año 2020, situándose la media del último año entorno a 650. Esta circunstancia es importante ya que se ha de vigilar el tratamiento que se hace con los valores de la columna `popularity`.

Además de `popularity`, el *DataFrame* `df_movies` incluye las variables relacionadas directamente con el ***_éxito_***, que es el objetivo principal del análisis. Como punto de partida se visualizarán las variables `avg_rating`, `num_votes`, `revenue`, y `popularity`, así como las relaciones entre ellas. Existen varias formas de hacerlo, pero *seaborn* permite mostrar toda la información en un solo gráfico.

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 2</font>

Generar un figura con una matriz de gráficas en las que se muestre, para cada par de variables (de entre las cuatro mencionadas), una gráfica de dispersión; y para cada variable, un histograma.  Esta figura se puede hacer de manera sencilla mediante `seaborn.pairplot()` o `seaborn.PairGrid()`. Debido a que el conjunto de datos contiene información sobre casi 4000 películas, es necesario hacer ciertos ajustes sobre las gráficas de dispersión, como por ejemplo utilizar un tamaño reducido o transparencia para las marcas. 


<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* En este ejercicio hay que seleccionar las columnas de interés, y pasar el *DataFrame* resultante  a la función de *Seaborn* (se puede hacer en la misma llamada).
* Mediante `corner=True` se puede mostrar solo una parte de la matriz (que es simétrica). A veces es más claro así.
* Por otra parte, es posible pasar argumentos a las gráficas de la diagonal con el diccionario `diag_kws={'arg1':value1, 'arg2':value2,...}`; y a las gráficas bivariadas mediante el diccionario `plot_kws={'arg1':value1, 'arg2':value2,...}`.

</details>   




In [ ]:
success_meas = ['avg_rating','num_votes', 'revenue', 'popularity'] # Columnas relativas a la valoración

# Completar

<!--comment -->

La gráfica anterior muestra que los valores de algunas variables (`num_votes`, `revenue`, y `popularity`) están muy sesgados, de modo que la mayoría se situan en la parte más baja del rango. Aún así, se percibe cierta relación entre `avg_rating` y `num_votes`, ya que parece que la gente tiende a votar más las películas que le gustan. También se percibe una relación similar, aunque menos fuerte, entre `avg_rating` y `revenue`. 



Aunque a priori no parece conveniente eliminar las películas con valores altos en `popularity`, los _outliers_  dificultan la visión tanto del histograma correspondiente como de las gráficas de dispersión.  Por tanto, se ha de repetir la gráfica, pero descartando las películas en las que `popularity` toma un valor extremadamente alto. Anteriormente se ha visto que, si bien la popularidad crece de forma pronunciada, la media en el último año se situa ligeramente por encima de 650. También que la diferencia crece de manera dramática a partir del cuantil 0.995. Por tanto, se almacenará este valor en la variable `pop_outlier_thresh`, y se utilizará como umbral.

In [ ]:
pop_outlier_thresh = df_movies['popularity'].quantile(0.995)
pop_outlier_thresh

Repetir la gráfica elaborada en el ejercicio uno, pero descartando las filas correspondientes a valores de `popularity` mayores que `pop_outlier_thresh` (sin eliminarlos del *DataFrame* `df_movies`).

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* El ejercicio es similar al ejercicio anterior, pero hay que hacer una selección condicional de las filas cuya popularidad está por debajo del umbral `pop_outlier_thresh` y utilizar el *DataFrame* resultante para hacer la gráfica.
</details>   





In [ ]:
# Completar

<!--comment -->

El descarte de los *outliers* permite ver ahora todas las gráficas con claridad. Parece que `num_votes`, y en mayor grado `revenue` guardan una ligera relación con `popularity`. Se aprecia que también que `avg_rating` se comporta de manera distinta, y que la relación con `popularity` es prácticamente nula. En cualquier caso, debido al gran número de puntos, y aunque se pueden intuir algunas tendencias, es difícil establecer hacer una valoración clara. En este caso, el coeficiente de correlación entre las variables puede ser proporcionar una visión más precisa. 




Obtener el coeficiente de correlación entre los pares de variable. Repetir la operación para los datos con y sin _outliers_ (en `popularity`) y comparar.


<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>

* Se puede utilizar `DataFrame.corr()`.
* Solo hay que seleccionar las columnas de interés (en `success_meas`) y aplicar el método al *DataFrame* resultante.
</details>   

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> `DataFrame.corr()` devuelve, a su vez, un *DataFrame*. El método `DataFrame.style.background_gradient()` muestra el conjunto de datos resultante con colores que facilitan su interpretación. 
</div>

Muestra la correlación incluyendo los *outliers*.

In [ ]:
# Completar

<!--comment -->

Muestra la correlación descartando los *outliers* en función del valor `pop_outlier_thresh`.

In [ ]:
# Completar

<!--comment -->

Además, se considera que el presupuesto de las películas (`budget`) podría tener una incidencia definitiva en su difusión y, por tanto, en su éxito. Por ello se va a estudiar su relación con algunas de las variables anteriores. 

Obtener la correlación de `budget` con las variables `avg_rating`, `num_votes`, `revenue`, y `popularity`. 



<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Para hacer este ejercicio, solamente hay que acceder a las 5 columnas de interés y obtener la matriz de correlación. En este caso, solamente necesitamos los datos correspondientes a `budget` del resultado, por lo que se puede mostrar la columna correspondiente del *DataFrame*.
</details>   

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> A menos que se indique explícitamente, se trabajará con el conjunto de datos completos (sin eliminar _outliers_).
</div>

In [ ]:
success_meas.append('budget')

# Completar

Efectivamente, el presupuesto tiene una correlación relativamente alta con los ingresos, a pesar de no tenerla (incluso es ligeramente negativa) con las valoraciones. Esto refleja la importancia de la promoción. También que las valoraciones tienden a ser más bajas (muy ligeramente) para películas de alto presupuesto (¿el público exige más?). 

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<br>
Los datos obtenidos muestran algunos factores de interés:

* Cuando se consideran todas las películas, la correlación más alta se da entre el número de votos (`num_votes`) y los ingresos de la película (`revenue`), lo cual es esperable: cuanta más gente ve una película, más gente tiende a votarla. La correlación entre la valoración de la película (`avg_rating`) y los ingresos (`revenue`) es nula. Esto hace pensar que la gente va a ver las películas por la promoción y la espectativa que levantan, independientemente de las críticas o las valoraciones. 


* Si se eliminan los _outliers_ de `popularity`, la relación entre esta variable y las demás cambia ligeramente. En concreto la popularidad ( `popularity`) parece relacionada de manera algo más fuerte con los ingresos (`revenue`). Esto bien podría deberse a que la popularidad de las novedades es mayor, mientras que la recaudación es menor por haber estado menos tiempo en taquilla. Al descartarlas, este fenómeno no afecta tanto.


* La valoración (`avg_rating`)  tiene cierta relación positiva con el número de votos (`num_votes`). Parece que la gente tiende a votar para expresar una valoración positiva.


* El presupuesto (`budget`) está muy relacionado con el beneficio (`revenue`).

Llegado a este punto, y considerando el objetivo inicial del estudio, nos centraremos principalmente en la variable `revenue` para medir la repercusión de la película (es proporcional a la gente que la ha visto), aunque también se estudiarán en algunos casos la valoración y número de votos.

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#00586D"> 2. Presupuesto</font>
<br>



<a id="section21"></a>
### <font color="#00586D"> 2.1 Presupuesto / ingresos  </font>
<br>

Los datos anteriores mostraban que la correlación entre el presupuesto (`budget`) y los ingresos (`revenue`) es positiva. Es posible estudiar con más en detalle esta relación. Un recurso interesante en este sentido es un gráfico de tipo `sns.joinplot()`, que dibuja una gráfica de dispersión de las dos variables, la distribución marginal de cada una de ellas y,  mediante el parámetro `kind='reg'` puede añadir la recta correspondiente a un modelo de regresión. 



#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 3</font>



Construir y analizar una gráfica `sns.joinplot()` con una recta de regresión entre las variables `budget` y `revenue`. 

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Este tipo de gráficas permite pasar argumentos a la gráfica de dispersión. Cuando `kind='reg'`, `scatter_kws={'s':5, 'alpha':0.2}` haría que se dibujen los puntos con tamaño 5 y un gran nivel de transparencia, lo que permite apreciar densidades cuando el número de puntos es grande.
</details>  

In [ ]:
# Completar

<!--comment -->

Se aprecia cierta relación lineal entre `budget` y `revenue`. Sin embargo, la gráfica no permite determinar si esta relación es significativa.


Estimar un modelo de regresión lineal con `statsmodels` y mostrar los resultados. ¿Qué valor tiene el $R^2$? ¿Qué significa? ¿Se puede afirmar que existe una relación entre el presupuesto y el beneficio? ¿Por qué? 

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
*  En las transparencias sobre regresión lineal (del módulo III) aparece un ejemplo de regresión univariada con `statsmodels`.
*  Existen valores perdidos en la columna `budget`. Esto implica que hay que fijar el parametro `missing='drop'` para `sm.OLS`.
* Se han de proporcionar respuestas a las preguntas en la celda correspondiente.
</details>  


In [ ]:
import statsmodels.api as sm

# Completar

<!--comment -->

`Comentar el resultado (contestar a las preguntas) en esta celda`.


<br><br><br>

<!--comment -->

Aunque el modelo establece que existe una relación lineal, se puede apreciar que está condicionada por los valores extremos de  `revenue`. Además parece que la varianza en esta variable aumenta cuando lo hace `budget`. 



Comprobar esta circunstancia con una gráfica de tipo `sns.residplot()`.

In [ ]:
# Completar

<!--comment -->

Efectivamente, la varianza de `revenue` aumenta conforme aumenta `budget`, es decir, no se cumple la condición de homocedasticidad, por lo que los intervalos de confianza y p-valores asociados al modelo de regresión ***no tienen validez estadística*** aunque sí permiten intuir la relación entre las variables.

Por otra parte, llama la atención que existen varios puntos que rompen claramente las tendencias, ya que existen varias películas que obtuvieron unos ingresos significativamente por encima de las demás, junto con algún fracaso que, pese al alto presupuesto, no generó los ingresos esperados. 

Obtener la información (título, presupuesto y fecha de estreno) relativa a las cinco películas con más presupuesto.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Existe un método que permite obtener *directamente* los *n* mayores elementos dada una columna.
</details> 

In [ ]:
# Completar

<!--comment -->

Obtener la información (título, ingresos y fecha de estreno) relativa a las diez películas con más ingresos.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Existe un método que permite obtener *directamente* los *n* mayores elementos dada una columna.
</details>  


In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Hay algunas películas entre las cinco con mayor presupuesto que no aparecen entre las diez con más ingresos.

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section22"></a>
### <font color="#00586D"> 2.2 La fecha de estreno como factor de interés  </font>
<br>

Aunque la correlación entre presupuesto y los ingresos corresponde con lo que cabe esperarse (más presupuesto implica también más publicidad), hay que considerar que ambos factores dependen fuertemente del año de estreno, ya que se están manejando datos que corresponden a épocas distintas. Para tener una idea de la importancia de este factor, es necesario visualizar como evolucionan ambas cantidades a lo largo de los años.




#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 4</font>

Agrupar las películas en función del año de estreno, y calcular la media de presupuesto e ingresos para cada grupo. Para ello, se utilizará la función `DataFrame.groupby()`, y se almacenará el resultado (la media para `budget` y para `revenue` de cada año) en un *DataFrame* que se denominará `df_bud_rev_year`.


<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Como se ha visto anteriormente, los valores de tipo `TimeStamp` tienen un campo, denominado `year` que contiene el año. 
* Mediante `df_movies['release_date'].dt.year` se puede acceder a la serie de años y utilizarla para hacer la agrupación (recordad que a `DataFrame.groupby()` se le puede pasar una *Serie* en lugar de un nombre de columna).  
* Para que el resultado sea correcto, la agrupación se ha de hacer sobre `df_movies.dropna()`.
</details>  

In [ ]:
df_bud_rev_year = # Completar

<!--comment -->

 Dibujar la gráfica a partir de la información (dos columnas) que contiene el `DataFrame` `df_bud_rev_year`.
 
 
 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Se puede hacer directamente con `sns.lineplot()`
</details>  

In [ ]:
# Completar

<!--comment -->

En la gráfica se pueden visualizar varios fenómenos:
* Tanto los presupuestos como los beneficios crecen cada año.
* La línea de beneficios es más irregular que la de presupuesto.
* Los beneficios parecen crecer más que los presupuestos, sobre todo en los últimos años.
* Los beneficios cayeron de forma dramática el primer año de la pandemia. 
* Se aprecian seis anomalías importantes anteriores a 1980, en las que se aprecia que los ingresos medios se separan ampliamente de la tendencia. 



Obtener los años a los que corresponden estas entradas. Para ello, pueden obtenerse los 6 mayores valores anteriores a 1980, en el _DataFrame_ `df_bud_rev_year`.

 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* El *DataFrame* está indexado y ordenado por años, por lo que se puede acceder directamente a las filas de interés.
* Se trata de obtener 6 filas con mayor valor en la columna `revenue`.
</details>  

In [ ]:
# Completar

<!--comment -->

En 1937, 1939, 1942, 1951, 1965 y 1977 se producen las alteraciones más importantes de la serie.



Extraer los datos de las películas correspondientes a estos años y almacenarlos en un _DataFrame_ denominado `df_anomalies`. Dibujar un gráfico de barras horizontal con los ingresos de cada película. Las etiquetas en el eje `y` deben contener el título de la películas y, entre paréntesis, el año de estreno.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Para hacer este ejercicio, primero hay que crear  `df_anomalies` mediante selección condicional con `Series.dt.year` y utilizando el método `Series.isin()`.
* Para hacer la gráfica se puede utilizar el método `plt.barh()` de *matplotlib*.
* Cuando hay muchas categorías, o los nombres de éstas son largos, es conveniente que la gráfica sea horizontal.
</details>  

In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Puede apreciarse que estas anomalías se deben a películas concretas como "_Gone with the wind_" ("Lo que el viento se llevó"), "_Alice in Wonderland_" ("Alicia en el país de las maravillas") y, sobre todo, "_Star Wars_". También hay otras como "_Close Encounters with the Third Kind"_ ("Encuentros en la tercera fase"), o algunas otras de animación como "Bambi" y "_Snow White and the Seven Dwarfs_" ("Blancanieves y los siete enanitos").

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---


### <font color="#00586D"> Conclusión</font>
<br>

Como resumen hasta ahora puede establecerse que, efectivamente, el presupuesto está muy relacionado con los ingresos. Con respecto a la valoración, sin embargo, se había visto en la sección anterior que no parece que haya una relación relevante, lo que constituye el hallazgo que, a priori, podría resultar contrario a nuestra intuición. 

Además, hay una variable _latente_ que condiciona los ingresos: la fecha de estreno. Para estudiar la repercusión de otros factores se hace necesario tener en cuenta esta circunstancia. Existen varias posibilidades. 

* Corregir los ingresos y beneficio en función del año, eliminando el efecto de la tendencia.
* Estudiar distintos periodos de tiempo por separado. 
* Hacer el estudio para franjas de prespuesto específicas. 
* Calcular el porcentaje o ratio de ingresos con respecto al presupuesto. 

Debido esto, y al objetivo del estudio (producción de una película de presupuesto relativamente bajo), se considerarán películas más actuales. Además, desde el año 2000 en adelante se observa que el crecimiento del presupuesto es algo más lento, por lo que se ha decidido centrar el estudio en películas estrenadas a partir de esa fecha. Además, se trabajará también con el ratio recaudación/presupuesto.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Cuando se trabaja con datos temporales, se suele eliminar la tendencia o la estacionalidad. En este proyecto no se trabajará a ese nivel, ya que estos conceptos se tratarán en el módulo VII.
</div>

In [ ]:
df_movies = df_movies[df_movies['release_date']>='2000']

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#00586D"> 3. Géneros</font>
<br>

En la base de datos original, `df_movies`, los géneros de cada película aparecen como una lista separada por comas en la columna `genres`. Para poder estudiar la repercusión de este factor, es necesario llevar a cabo una **transformación** que permita operar de manera ágil. Dado que el número de géneros es limitado y reducido, es posible utilizar variables identificadoras (_dummy_).  _Otra posibilidad_ consiste en crear un conjunto de datos auxiliar, `df_genres`, en formato _long form_, en el que cada fila corresponda a un par película-género, y hacer las uniones correspondientes con`df_movies`.  

<font size=2 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i> </font> Podéis ir ejecutando cada sentencia por separado y ver el resultado para entender mejor el proceso.

In [ ]:
# Crea las variables dummy para indicar qué género aparece en cada película.
df_genres = df_movies['genres'].str.get_dummies(',').astype(bool)
# Almacena los nombres de los  géneros en una lista.
genres = df_genres.columns.to_list()
# Convierte la matriz a formato "long form"
df_genres = (df_genres.reset_index()
                      .melt(id_vars='movie_id', value_vars=genres, var_name='genre', value_name='ind')
                      .set_index('movie_id'))
# Filtra 
df_genres = df_genres[df_genres['ind']==True]
# Borra la columna indicador
del df_genres['ind']

# Muestra las entradas correspondientes a la película "Avatar" (Adventure, Action, Fantasy)
df_genres.loc['tt0499549']

Además, se descartarán las columnas de `df_movies` que no se utilizarán.

In [ ]:
df_movies = df_movies[['title','avg_rating', 'num_votes', 'budget', 'revenue', 'popularity', 'release_date']]

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 5</font>

A partir del *DataFrame* `df_genres`, es posible visualizar con qué frecuencia aparecen los géneros.  Dibujar un gráfico de barras que muestre el número de entradas correspondientes a cada valor de `genre` en el *DataFrame* `df_genres`.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Se puede hacer la gráfica directamente con `Seaborn.catplot()`.
* Cuando existen muchas categorías o el nombre es largo, puede convenir que el gráfico sea horizontal. Para hacerlo, hay que representar la variable categórica en el eje *y*.
</details>  

In [ ]:
# Completar

<!--comment -->

Parece que el drama es, de largo, el género que tratan más películas. Le siguien la acción y la comedia.



 Mostrar la distribución de popularidad por género con un gráfico de cajas (horizontal). Esto requiere utilizar información de `df_genres` (género) y de `df_movies` (popularidad).
 
 
 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Analizar la información por género implica fundir el *DataFrame* `df_genres`con `df_movies['popularity']`, preservando todas las entradas de `df_genres`(es necesario fundir por índice con `on='movie_id'`, y especificar adecuadamente el valor para el parámetro `how`).
    
* Los outliers de `popularity` hacen que no se vea bien la gráfica, por lo que es conveniente hacer selección condicional sobre `df_movies` (utilizando `prop_outlier_thresh`) antes de la fusión. 
    
* El nuevo *DataFrame* se puede visualizar como un gráfico de cajas directamente con *Seaborn*.
    
* Por otra parte, para mejorar la visualización se puede jugar con el aspecto (`aspect`), el tamaño de las líneas `linewidth`, o el tamaño de los puntos que representan los *outliers* (`fliersize`).
    
* No es necesario hacerlo todo en una llamada (aunque se puede), y se puede crear un *DataFrame* intermedio.
</details>  

In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 6</font>

Para explorar la relación entre presupuestos e ingresos en función del género, debido a que las distribuciones están sesgadas, también se utilizarán gráficas del tipo `sns.boxplot()`. Como paso previo, y para facilitar la exploración, se pueden ordenar los géneros en función de la ___mediana___ de los ingresos obtenidos (`revenue`) por las películas correspondientes. Este orden se almacenará en la lista `ordered_genres`.

 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* El primer paso consiste en fundir `df_genres` con `df_movies['revenue']`.
    
* El *DataFrame* resultante se ha de agrupar según la columna `genre`. 
    
* Posteriormente se ha de acceder a la columna `'revenue'` de la agrupación, y devolver la mediana. 
    
* La serie resultante se puede ordenar y posteriormente se puede devolver el índice de la misma (que preserva el orden).
    
* No es necesario hacerlo todo en una llamada (aunque se puede)
</details> 

In [ ]:
ordered_genres = # Completar
print(ordered_genres)

<!--comment -->

 Dibujar un gráfico de cajas y bigotes con el beneficio por género, utilizando el orden entre géneros almacenado en `ordered_genres`.
 
  <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* El primer paso consiste en fundir `df_genres` con `df_movies['revenue']`.
    
* Se puede hacer la gráfica directamente con `Seaborn.catplot()` a partir del *DataFrame* resultante.
    
* Para pasar el orden de las cajas puede utilizarse el parámetro `order=ordered_genres`.
    
* Por otra parte, pueden cambiarse los colores de la gráfica, por ejemplo, con `palette="flare"`, que es una paleta secuencial.
    
* También se deben rotar las etiquetas del eje *x* con `plt.xticks()`.
    
* No es necesario hacerlo todo en una llamada (aunque se puede)
</details> 

In [ ]:
# Completar

<!--comment -->

Puede apreciarse que las películas de animación, aventuras, familia y ciencia ficción, son los géneros que más recaudan. Los que menos, las noticias, los deportes y los documentales. Es curioso, aunque puede estar relacionado, que el drama es el género más frecuente y es uno de los que menos recauda.
<br>




Anteriormente se comprobó la importancia del presupuesto en la recaudación. Por tanto, es interesante también estudiar si la relación de las recaudaciones por género se refleja también en los presupuestos.   Elaborar una gráfica similar a la anterior en la que se muestre el presupuesto por géneros. Para comparar, puede ser de utilidad mantener el mismo orden en la representación utilizado anteriormente (la mediana de los ingresos).


  <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Es ***idéntico*** al anterior. Solo hay que utilizar la columna `revenue` por `budget` en la gráfica.
    
* En este caso, el uso de la paleta `flare` puede distorsionar la percepción de la grafica ya que hace percibir más orden del que realmente hay. Utilizar un color solamente. Por ejemplo, un tono de gris claro con `color='.95'`.
</details> 

In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Se aprecian algunos cambios menores.  No obstante, la tendencia general, es que los géneros que producen más ingresos son también los que más presupuesto reciben. 

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#00586D"> 4. Personas</font>
<br>

Una vez estudiados los géneros, va a estudiarse si el director y el reparto tienen relación con la rentabilidad de las películas. El _DataFrame_ `df_credits` contiene la relación de personas que participan en cada película y en calidad de qué. Por otra parte, el _DataFrame_ `df_people` contiene la información relativa a cada persona, incluyendo su nombre.







<a id="section41"></a>
### <font color="#00586D"> 4.1 Directores</font>
<br>

Interesa obtener información sobre los ingresos que producen los directores,  sobre su popularidad y la relación de ambos factores.

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 7</font> 


Elaborar una gráfica `sns.jointplot()` para comprobar si existe correlación entre los ingresos de las películas y la popularidad de su director. Esto implica utilizar datos de `df_credits`, de `df_people[popularity]` y de `df_movies['revenue']`) por lo que hay que hacer dos fusiones. El resultado de estas se debe almacenar en un *DataFrame* denominado `df_dir_pop_rev`.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* En primer lugar, hay que hacer selección condicional sobre `df_credits`, de modo que se seleccionen las filas en las que `rol` sea `director`.
* El resultado se ha de fundir con `df_people['popularity']` utilizando `people_id` como clave de unión.
* El resultado se ha de fundir a su vez con `df_movies['revenue']` utilizando `movie_id` como clave de unión.
* En los dos casos hay que ser consciente de cómo (`how`) se hace la unión. Se ha de tener en cuenta que muchas películas en `df_credits` no están en `df_movie`. Esto puede (según se haga la unión) generar *NaN* que hay que eliminar.
* La gráfica se puede mostrar directamente. Conviene utilizar transparencia para poder apreciar densidades.
</details>  


In [ ]:
df_dir_pop_rev = # Completar

In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Puede apreciarse que la relación entre popularidad del director y el beneficio es casi nula.

In [ ]:
df_dir_pop_rev[['popularity','revenue']].corr()

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section42"></a>
### <font color="#00586D"> 4.2 Reparto</font>
<br>


Una vez estudiado el papel del director, se procederá de igual manera con el reparto. 

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 8</font> 

Elaborar una gráfica `sns.jointplot()` para comprobar si existe correlación entre los ingresos de las películas y la popularidad de su reparto. El resultado de estas se debe almacenar en un *DataFrame* denominado `df_cast_pop_rev`.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* El ejercicio es similar al anterior. Solo hay que cambiar el rol a `cast` y nombrar el *DataFrame* resultante `df_cast_pop_rev`. 

</details>  


In [ ]:
df_cast_pop_rev = # Completar

In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

In [ ]:
df_cast_pop_rev[['popularity','revenue']].corr()

Se aprecia también que, aunque influye positivamente, no es determinante porque casi es nula.


En definitiva, y según se aprecia en las gráficas, el principal factor en relación a la recaudación es el presupesto, mientras que otros como el director o los actores sean irrelevantes (salvo en casos muy específicos, se intuye).


Aunque podría profundizarse en el análisis, esto se hará posteriormente teniendo en cuenta el objetivo con el que se plantea la exploración.

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section5"></a>
## <font color="#00586D"> 5. Películas de bajo presupuesto</font>
<br>

En el objetivo de estudio que se ha establecido inicialmente, se parte de la base de que se dispone de un presupuesto limitado, por lo que se va a estudiar también la relación entre género e ingresos en este supuesto, considerando solamente aquellas películas cuyo presupuesto se encuentra en el 20% más bajo.

A continuación se almacenan las películas en las que se centrará el estudio (posteriores al año 2000 y de bajo presupuesto) en `df_movies_low`.

In [ ]:
low_budget = df_movies['budget'].quantile(0.20)
df_movies_low = df_movies[df_movies['budget']<low_budget]
len(df_movies_low)

Aunque es algo que se puede gestionar "sobre la marcha", retener solamente las películas de interés en `df_genres` facilita las cosas. En este caso se seleccionan y se almacenan en `df_genres_low`.

In [ ]:
df_genres_low = df_genres.loc[df_movies_low.index]

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 9</font> 

Recalcular el orden de los géneros en función de la mediana los ingresos y construir una gráfica de cajas con los datos almacenados en el *DataFrame* `df_movies_low`.

<br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Se trata de repetir el primer apartado del ejercicio 6, pero con `df_genres_low` y `df_movies_low`.

</details>  


In [ ]:
ordered_genres = # Completar
print(ordered_genres)

<!--comment -->

In [ ]:
# Completar


<!--comment -->

Puede apreciarse un cambio claro de tendencia. Cuando los prespuestos son relativamente bajos, géneros como el misterio y el terror pasan a ser más rentables.  También algún otro género como las noticias o historia. Llama también la atención que, géneros como la animación o fantasía, que son los más rentables en el caso general, no son rentables cuando los presupuestos son bajos.
<br>

Como los géneros más rentables son el terror, y el misterio (si se descartan las noticias, que solo hay una película), se seleccionarán las películas correspondientes a estos, además del _thriller_ (esta última por estar relacionada y mostrar también algunos valores altos) para estudiarlas en más profundidad. 



 Eliminar de `df_movies_low` las películas que no correspondan a los géneros `Horror`, `Thriller` o `Mystery`.
 
 
 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* En realidad se trata de seleccionar las filas de interés y guardar el resultado en `df_movies_low`.
* Se trata de hacer selección condicional con `Series.isin()` sobre `df_genres_low`. El resultado contiene los índices de las películas de interés. 
* Los *valores únicos* de este índice se pueden utilizar para acceder a las filas de `df_movies_low`.

</details>  


In [ ]:
# Completar

df_movies_low.head()

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Una vez reducido el conjunto de películas a las que resultan de interés para el contexto planteado, se obtendrán la lista de directores y actores ligados a una mayor recaudación. 

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 10</font> 

Crear un *DataFrame* denominado `best_directors_low` con los datos (`people_id`, `revenue`, `name`, `popularity` y `gender`) de los diez directores para los que la recaudación media de sus películas es mayor.


 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Este ejercicio implica obtener datos de los tres *DataFrame* 
* Hay que filtrar los créditos correspondientes a director
* El resultado se ha de fundir con `df_movies_low['revenue']` 
* El *DataFrame* resultante se ha de agrupar en función del `people_id` y obtener el valor medio de `revenue` 
* Se pueden obtener directamente las diez filas o elementos con mayor valor de `revenue`, y fundir el resultado con `df_people` para disponer también de los nombres y otros datos.
</details>  

In [ ]:
# Completar

best_directors_low

<!--comment -->

Crear una gráfica de barras (horizontal) con los valores `revenue` para cada director en `best_directors_low`.

In [ ]:
# Completar

<!--comment -->

A continuación se muestra la película del director que ha obtenido más beneficios que, según parece, también es protagonista de la película.

In [ ]:
# people_id del director
best_director_id = best_directors_low.iloc[0].name
# Créditos de las películas de ese director y que estén en df_movies_low
movies_bd = (df_credits[(df_credits['people_id']==best_director_id) & 
                       (df_credits['movie_id'].isin(df_movies_low.index))]
                       ['movie_id'])
# Películas
df_movies.loc[movies_bd]

Podemos mostrar el cartel de la película que ha catapultado al éxito (al menos económico) a este director.

In [ ]:
from IPython.display import Image

Image('http://image.tmdb.org/t/p/w185//8ouiyGYcv3E1JPrPrC7pLjxlDhC.jpg')

Un modo quizá más justo de calcular la rentabilidad de una película es el ratio `revenue`/`budget` (para obtener este dato, es necesario considerar solamente las películas con `revenue>0` y `budget>0`, pero en primer apartado se eliminaron las que no cumplían la condición). Obtener los 10 directores que han generado un mayor beneficio según este criterio. 

  
Mostrar una gráfica, similar a la anterior, con esta información. En lugar de utilizar `revenue`, hay que calcular primero una nueva columna, denominada `rent` con la beneficio/presupuesto, y mostrar los datos corrspondientes a esta nueva columna.

 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Se trata de ***repetir*** el primer apartado del ejercicio, pero creando una nueva columna denominada `df_movies_low['rent']` el ratio entre recaudación y presupuesto, y utilizando  esta columna para crear otra vez `best_directors_low`.
</details>  

In [ ]:
# Completar

best_directors_low

<!--comment -->

Crear una gráfica de barras (horizontal) con los valores `rent` para cada director en `best_directors_low`.

In [ ]:
# Completar

<!--comment -->

En este sentido, hay un director y película que destaca sobre muy por encima de los demás.  

In [ ]:
# people_id del director
best_director_id = best_directors_low.iloc[0].name
# Créditos de las películas de ese director y que estén en df_movies_low
movies_bd = (df_credits[(df_credits['people_id']==best_director_id) & 
                       (df_credits['movie_id'].isin(df_movies_low.index))]
                       ['movie_id'])
# Películas
display(df_movies.loc[movies_bd])

Image('http://image.tmdb.org/t/p/w185/1bjA7de4O0NhMsaOqwvrecophUs.jpg')

Una vez estudiados los directores, procederemos a estudiar el reparto.  En este caso, se comenzará qué actores o actrices participan en películas con más ingresos.  


 Obtener los nombres de los 20 actores/actrices cuyas películas generan, por término medio, una mayor cantidad de ingresos. Almacenar el resultado en un *DataFrame* denominado `best_cast_low`.
 
 
  <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* El ejercicio es similar al anterior, pero hay que cambiar el `rol` y seleccionar las entradas correspondientes a `cast`. 
</details>  
  


In [ ]:
# Completar

best_cast_low

<!--comment -->

Crear una gráfica de barras (horizontal) con los valores `revenue` para cada director en `best_cast_low`.

In [ ]:
# Completar

<!--comment -->

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

En ejercicio anterior se muestra un reparto más o menos equitativo entre actores y actrices para los 10 miembros del reparto que están ligados a mayor beneficio.  Sin embargo, se estudiará algo más este factor (género).

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 11</font> 

Crear un *DataFrame* denominado `cast_low`, similar al anterior, con el beneficio medio (`revenue`) asociado a cada actor, su nombre (`name`), popularidad (`popularity`) y género (`gender`). 


 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Es similar al anterior, pero en lugar de acceder a los mayores valores, se devuelven todos las filas.
</details>  

In [ ]:
cast_low = # Completar
cast_low

<!--comment -->

A partir del *DataFrame* `cast_low`, obtener un gráfico de barras mostrando el número actores de cada género.

In [ ]:
# Completar

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Se aprecia que el número de actores es mayor que el de actrices. Cabe preguntarse si hay algún tipo de diferencia en la recaudación vinculada al género de los actores. 

#### <font  color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> Ejercicio 12</font> 

Mostrar la distribución de `cast_low` para los géneros 1 (mujer) y 2 (hombre). 

 <br>

<details>    
<summary>
    <font size=3 color="#00586D"> <i class="fa fa-info-circle"  aria-hidden="true"></i></font>
</summary>
    
* Se puede hacer un `sns.displot` con `cast_low` filtrando previamente por género para que solo contemple esos dos (mayoritarios).
* El color se puede utilizar para distinguir ambas distribuciones.    
</details>  

In [ ]:
# Completar

<!--comment -->

En principio se aprecia alguna diferencia en las distribuciones, pero no está claro que haya una diferencia en la recaudación media. Se puede verificar este hecho mediante un test U de Mann-Whitney. Si fijamos un nivel de significancia del 95% ¿puede afirmarse que son iguales los sueldos medios?

In [ ]:
from scipy.stats import mannwhitneyu

stat, p = # Completar
print(f'stat={stat:.3f}, p={p:.3f}')

`Comentar el resultado (contestar a las preguntas) en esta celda`.


<br><br><br>

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Se percibe que, a pesar de que aparecen más actores que actrices, parece que en las películas con los beneficios más altos no hay diferencia o incluso aparecen más actrices que actores.  El test refleja que en general no hay diferencias en las recaudaciones de las películas en que participan unos y otros.

<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<a id="section6"></a>
## <font color="#00586D"> 6. Conclusión</font>
<br>

En este trabajo se partía de dos bases de datos de películas y gente, y el objetivo era estudiar qué factores están más relacionados con el éxito de una película, que se mide por los ingresos que genera. Desde el principio, ha quedado claro que el presupuesto es el factor más importante,  que los géneros en que más dinero se gasta en la producción son la animación, fantasía, etc., y que éstos son también los que más recaudación obtienen.  También se ha comprobado que la información relativa a popularidad de directores y actores __no__ parece un factor muy relevante. 
<br>

No obstante se ha limitado el estudio al periodo actual y al (relativamente bajo) presupuesto disponible. En ese sentido, se ha comprobado que las películas más rentables son aquellas que pertenecen a los géneros de terror, suspense, etc. También se ha descubierto que películas como "Paranormal Activity" generan unos ingresos extraordinariamente altos en proporción con la inversión.  
<br>


<div align="right">
<a href="#indice"><font size=5 color="#00586D"><i class="fa fa-arrow-circle-up" aria-hidden="true"></i></font></a>
</div>

---

<div align="right">
<a href="#indice"><font size=6 color="#00586D"><i class="fa fa-coffee" aria-hidden="true"></i></font></a>
</div>